In [1]:
import os
from pm4py.objects.log.importer.xes import factory as xes_importer
import pandas as pd
import numpy as np

1) Set of all tasks and frequencies

In [2]:
log_tab = [[0,1,2,3,5,6,8],[0,1,2,4,5,7,8],[0,1,2,1,2,3,5,6,8],[0,0,1,2,4,5,7,8],[0,0,1,2,3,5,6,8],[0,0,1,2,1,2,4,5,7,8],[0,1,2,1,2,1,2,4,5,7,8],[0,1,2,1,2,1,2,3,5,6,8],[0,1,2,1,2,4,5,7,8],[0,0,1,2,1,2,3,5,6,8],[0,1,2,1,2,1,2,1,2,3,5,6,8],[0,0,1,2,1,2,1,2,4,5,7,8],[0,0,0,1,2,4,5,7,8],[0,0,0,1,2,1,2,3,5,6,8],[0,0,0,1,2,3,5,6,8],[0,0,0,0,1,2,4,5,7,8]]
#log_tab = [[0,1,2,3],[0,2,1,3],[0,4,3],[0,1,2,4,3],[0,4,2,1,3],[0,3]]
max_task = 0

for row in log_tab:
    if (max(row) > max_task):
        max_task = max(row)

print(log_tab)

[[0, 1, 2, 3, 5, 6, 8], [0, 1, 2, 4, 5, 7, 8], [0, 1, 2, 1, 2, 3, 5, 6, 8], [0, 0, 1, 2, 4, 5, 7, 8], [0, 0, 1, 2, 3, 5, 6, 8], [0, 0, 1, 2, 1, 2, 4, 5, 7, 8], [0, 1, 2, 1, 2, 1, 2, 4, 5, 7, 8], [0, 1, 2, 1, 2, 1, 2, 3, 5, 6, 8], [0, 1, 2, 1, 2, 4, 5, 7, 8], [0, 0, 1, 2, 1, 2, 3, 5, 6, 8], [0, 1, 2, 1, 2, 1, 2, 1, 2, 3, 5, 6, 8], [0, 0, 1, 2, 1, 2, 1, 2, 4, 5, 7, 8], [0, 0, 0, 1, 2, 4, 5, 7, 8], [0, 0, 0, 1, 2, 1, 2, 3, 5, 6, 8], [0, 0, 0, 1, 2, 3, 5, 6, 8], [0, 0, 0, 0, 1, 2, 4, 5, 7, 8]]


In [3]:
freq = [35,25,9,8,6,3,2,2,2,2,1,1,1,1,1,1]
#freq = [9,1,9,1,1,1]
print(freq)

[35, 25, 9, 8, 6, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1]


2) Connections matrix

In [4]:
patternA_list = []
connection = [[0 for x in range(max_task+1)] for x in range(max_task+1)] 
i = -1

for row in log_tab:
    i = i + 1
    for j in range(len(row)-1):
        tupl = (log_tab[i][j], log_tab[i][j+1])
        #if tupl not in patternA_list:
        x = log_tab[i][j]
        y = log_tab[i][j+1]
        connection[x][y] = connection[x][y] + freq[i]
        patternA_list.append(tupl)

print(connection) # dependency matrix

[[29, 100, 0, 0, 0, 0, 0, 0, 0], [0, 0, 130, 0, 0, 0, 0, 0, 0], [0, 30, 0, 57, 43, 0, 0, 0, 0], [0, 0, 0, 0, 0, 57, 0, 0, 0], [0, 0, 0, 0, 0, 43, 0, 0, 0], [0, 0, 0, 0, 0, 0, 57, 43, 0], [0, 0, 0, 0, 0, 0, 0, 0, 57], [0, 0, 0, 0, 0, 0, 0, 0, 43], [0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [5]:
def calculate_dependency(x,y):
    return (connection[x][y] - connection[y][x])/(connection[x][y] + connection[y][x] + 1)

3) Creating dependency matrix

In [6]:
dependency_matrix = [[0 for x in range(max_task+1)] for x in range(max_task+1)]  

for i in range(len(connection)):
    for j in range(len(connection)):
        dependency_matrix[i][j] = calculate_dependency(i,j)

print(dependency_matrix)

[[0.0, 0.9900990099009901, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-0.9900990099009901, 0.0, 0.6211180124223602, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, -0.6211180124223602, 0.0, 0.9827586206896551, 0.9772727272727273, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, -0.9827586206896551, 0.0, 0.0, 0.9827586206896551, 0.0, 0.0, 0.0], [0.0, 0.0, -0.9772727272727273, 0.0, 0.0, 0.9772727272727273, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, -0.9827586206896551, -0.9772727272727273, 0.0, 0.9827586206896551, 0.9772727272727273, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, -0.9827586206896551, 0.0, 0.0, 0.9827586206896551], [0.0, 0.0, 0.0, 0.0, 0.0, -0.9772727272727273, 0.0, 0.0, 0.9772727272727273], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.9827586206896551, -0.9772727272727273, 0.0]]


* Parameters

In [7]:
from ipywidgets.widgets import Label, Layout, Button, interact, interactive, IntSlider, Checkbox, VBox
from IPython.display import display
import numpy as np

dependency = IntSlider(description='Dependency:')
relative_to_best = IntSlider(description='Relative_to_best:')
length_one_loop = IntSlider(description='One_loop:')
length_two_loop = IntSlider(description='Two_loop:')
long_distance = IntSlider(description='Long_loop:')

all_task_connected = Checkbox(description='all_task_connected')
long_distance_dependency = Checkbox(description='long_distance_dependency')
ignore_two_loop = Checkbox(description='ignore_two_loop')

In [8]:
box = VBox(children=(dependency, relative_to_best, length_one_loop, length_two_loop, long_distance), layout=Layout(border='solid 2px gray', width='500px'))
box2 = VBox(children=(all_task_connected, long_distance_dependency, ignore_two_loop), layout=Layout(border='solid 2px gray', width='500px'))
button = Button(description = "Generate model")

def generate_model(b):
    dep = dependency.value/100#done
    rel_to_best = relative_to_best.value/100 #done
    len_one_loop = length_one_loop.value/100 #done
    len_two_loop = length_two_loop.value/100 #done
    l_distance = long_distance.value/100 #done
    a_task_connected = all_task_connected.value #done
    l_distance_dependency = long_distance_dependency.value #done
    i_two_loop = ignore_two_loop.value #done    
    
    print(rel_to_best)
        
    dependency_list = []
    dependency_matrix1 = [[0 for x in range(max_task+1)] for x in range(max_task+1)]  
    
    for i in range(len(connection)):
        for j in range(len(connection)):
            if(all_task_connected != 1):
                if(dependency_matrix[i][j] >= dep):
                    tupl = (i,j)
                    dependency_list.append(tupl)
                    dependency_matrix1[i][j] = dependency_matrix[i][j]
            else:
                if(dependency_matrix[i][j] > 0):
                    tupl = (i,j)
                    dependency_list.append(tupl)
                    dependency_matrix1[i][j] = dependency_matrix[i][j]
                    
    dependency_list = []
dependency_matrix1 = [[0 for x in range(max_task+1)] for x in range(max_task+1)]  

for i in range(len(connection)):
    for j in range(len(connection)):
        if(all_task_connected != 1):
            if(dependency_matrix[i][j] >= dependency):
                tupl = (i,j)
                dependency_list.append(tupl)
                dependency_matrix1[i][j] = dependency_matrix[i][j]
        else:
            if(dependency_matrix[i][j] > 0):
                tupl = (i,j)
                dependency_list.append(tupl)
                dependency_matrix1[i][j] = dependency_matrix[i][j]

print(dependency_list)
print(dependency_matrix1)
    print("Ok")

button.on_click(generate_model)


IndentationError: unexpected indent (<ipython-input-8-98ca8875894e>, line 51)

In [41]:
display(box)
display(box2)
display(button)

Button(description='Generate model', style=ButtonStyle())

0.77
Ok


0.9


* dependency, all_task_connected parameters

In [495]:
dependency_list = []
dependency_matrix1 = [[0 for x in range(max_task+1)] for x in range(max_task+1)]  

for i in range(len(connection)):
    for j in range(len(connection)):
        if(all_task_connected != 1):
            if(dependency_matrix[i][j] >= dependency):
                tupl = (i,j)
                dependency_list.append(tupl)
                dependency_matrix1[i][j] = dependency_matrix[i][j]
        else:
            if(dependency_matrix[i][j] > 0):
                tupl = (i,j)
                dependency_list.append(tupl)
                dependency_matrix1[i][j] = dependency_matrix[i][j]

print(dependency_list)
print(dependency_matrix1)

[(0, 1), (2, 3), (2, 4), (3, 5), (4, 5), (5, 6), (5, 7), (6, 8), (7, 8)]
[[0, 0.9900990099009901, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0.9827586206896551, 0.9772727272727273, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0.9827586206896551, 0, 0, 0], [0, 0, 0, 0, 0, 0.9772727272727273, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0.9827586206896551, 0.9772727272727273, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0.9827586206896551], [0, 0, 0, 0, 0, 0, 0, 0, 0.9772727272727273], [0, 0, 0, 0, 0, 0, 0, 0, 0]]


* relative_to_best parameter //TODO: moze trzeba tez takie zrobic dla kolumn, czyli dla relacji b,c=>d

In [496]:
for row in dependency_matrix1:
    maxi = row.index(max(row))
    threshold = row[maxi]*(1-relative_to_best)
    for elem in row:
        if(elem < threshold):
            x = dependency_matrix1.index(row)
            y = row.index(elem)
            dependency_matrix1[x][y] = 0
            tupl_tmp = (x, y)
            if tupl_tmp in dependency_list:
                dependency_list.remove(tupl_tmp)

print(dependency_list)
print(dependency_matrix1)

[(0, 1), (2, 3), (2, 4), (3, 5), (4, 5), (5, 6), (5, 7), (6, 8), (7, 8)]
[[0, 0.9900990099009901, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0.9827586206896551, 0.9772727272727273, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0.9827586206896551, 0, 0, 0], [0, 0, 0, 0, 0, 0.9772727272727273, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0.9827586206896551, 0.9772727272727273, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0.9827586206896551], [0, 0, 0, 0, 0, 0, 0, 0, 0.9772727272727273], [0, 0, 0, 0, 0, 0, 0, 0, 0]]


4) Short loops detection

* length_one_loop parameter

In [497]:
def calculate_shortloop(x,y):
    return connection[x][y]/(connection[x][y] + 1)

In [498]:
shortloop_matrix = [[0 for x in range(max_task+1)] for x in range(max_task+1)]  

for i in range(len(connection)):
    if(connection[i][i] != 0):
        var = calculate_shortloop(i,i)
        shortloop_matrix[i][i] = var
        if(var >= length_one_loop):
            tupl = (i,i)
            dependency_list.append(tupl)
    
print(dependency_list)
print(shortloop_matrix)

[(0, 1), (2, 3), (2, 4), (3, 5), (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (0, 0)]
[[0.9666666666666667, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]


* length_two_loop parameter

In [499]:
twoloop = [[0 for x in range(max_task+1)] for x in range(max_task+1)]
j = -1

for row in log_tab:
    j = j + 1
    for i in range(len(row)-2):
        first = row[i]
        second = row[i+1]
        third = row[i+2]
        if(first == third):
            twoloop[first][second] = twoloop[first][second] + freq[j]
            
print(twoloop)

[[5, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 30, 0, 0, 0, 0, 0, 0], [0, 30, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [500]:
def calculate_twoloop(x,y):
    return (twoloop[x][y]+twoloop[y][x])/(twoloop[x][y]+twoloop[y][x] + 1)

In [501]:
loop_list = []
twoloop_matrix = [[0 for x in range(max_task+1)] for x in range(max_task+1)]

for i in range(len(twoloop_matrix)):
    for j in range(len(twoloop_matrix)):
        var = calculate_twoloop(i,j)
        twoloop_matrix[i][j] = var
        if (ignore_two_loop == 0 and var >= length_two_loop and i!=j):
            tupl = (i,j)
            loop_list.append(tupl)

print(twoloop_matrix)
print(loop_list)

[[0.9090909090909091, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.9836065573770492, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.9836065573770492, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
[(1, 2), (2, 1)]


5) Long loops detection

* removed duplicates

In [502]:
longloop = []

for row in log_tab: 
    newlog_tab = []
    prev = row[0]
    newlog_tab.append(prev) 
    for x in row:
        if x != prev: 
            newlog_tab.append(x) 
            prev = x
    longloop.append(newlog_tab)

print(longloop)

[[0, 1, 2, 3, 5, 6, 8], [0, 1, 2, 4, 5, 7, 8], [0, 1, 2, 1, 2, 3, 5, 6, 8], [0, 1, 2, 4, 5, 7, 8], [0, 1, 2, 3, 5, 6, 8], [0, 1, 2, 1, 2, 4, 5, 7, 8], [0, 1, 2, 1, 2, 1, 2, 4, 5, 7, 8], [0, 1, 2, 1, 2, 1, 2, 3, 5, 6, 8], [0, 1, 2, 1, 2, 4, 5, 7, 8], [0, 1, 2, 1, 2, 3, 5, 6, 8], [0, 1, 2, 1, 2, 1, 2, 1, 2, 3, 5, 6, 8], [0, 1, 2, 1, 2, 1, 2, 4, 5, 7, 8], [0, 1, 2, 4, 5, 7, 8], [0, 1, 2, 1, 2, 3, 5, 6, 8], [0, 1, 2, 3, 5, 6, 8], [0, 1, 2, 4, 5, 7, 8]]


* create matrix of connections

In [503]:
longloop1 = [[0 for x in range(max_task+1)] for x in range(max_task+1)]

k = -1
for row in longloop:
    k = k + 1
    i = 0
    for col in row:
        i = i + 1
        for j in range(i,len(row)):
            if(col != row[j]):
                longloop1[col][row[j]] = longloop1[col][row[j]] + freq[k]

print(longloop1)

[[0, 130, 130, 57, 43, 100, 57, 43, 100], [0, 0, 168, 76, 54, 130, 76, 54, 130], [0, 38, 0, 76, 54, 130, 76, 54, 130], [0, 0, 0, 0, 0, 57, 57, 0, 57], [0, 0, 0, 0, 0, 43, 0, 43, 43], [0, 0, 0, 0, 0, 0, 57, 43, 100], [0, 0, 0, 0, 0, 0, 0, 0, 57], [0, 0, 0, 0, 0, 0, 0, 0, 43], [0, 0, 0, 0, 0, 0, 0, 0, 0]]


* creating vec of counted elems

In [504]:
counted_vec = []

for elem in range(max_task+1):
    counter = 0
    for row in log_tab:
        for col in row:
            if(col == elem):
                counter = counter + freq[log_tab.index(row)]
    counted_vec.append(counter)
    
print(counted_vec)

[129, 130, 130, 57, 43, 100, 57, 43, 100]


In [505]:
def calculate_longloop(x,y):
    return (longloop1[x][y])/(counted_vec[x] + 1)

* creating final longloop matrix

In [506]:
longloop_matrix = [[0 for x in range(max_task+1)] for x in range(max_task+1)]

for i in range(len(longloop1)):
    for j in range(len(longloop1)):
        var = calculate_longloop(i,j)
        if(var >= long_distance and long_distance_dependency == 0):
            longloop_matrix[i][j] = var

print(longloop_matrix)

[[0, 1.0, 1.0, 0, 0, 0, 0, 0, 0], [0, 0, 1.282442748091603, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]


6) Final data

In [507]:
print(dependency_list)
print(loop_list)

[(0, 1), (2, 3), (2, 4), (3, 5), (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (0, 0)]
[(1, 2), (2, 1)]


7) Creating a model

In [508]:
Tinput = []
for i in range(len(dependency_list)):
if log_tab[i][0] not in Tinput:
    Tinput.append(log_tab[i][0])

print(Tinput)

[0]


In [509]:
Toutput = []

for i in range(len(dependency_list)):
    if log_tab[i][len(log_tab[i])-1] not in Toutput:
        Toutput.append(log_tab[i][len(log_tab[i])-1])
        
print(Toutput)

[8]


In [510]:
footprint = np.zeros((max_task+1, max_task+1))

for i in dependency_list:
    x = i[0]
    y = i[1]
    footprint[x][y] = 1

for i in loop_list:
    x = i[0]
    y = i[1]
    footprint[x][y] = footprint[x][y] + 2
    
footprint[1][2] = 1
print(footprint)

[[1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [511]:
patternB_list = []

for i in range(len(footprint)):
    tab_tmp = []
    for j in range(len(footprint)):
        if(footprint[i][j] == 1):
            tab_tmp.append(j)
    if(len(tab_tmp) > 1):
        tab_tmp.append(i)
        tab_tmp = np.asarray(tab_tmp)
        patternB_list.append(tab_tmp)
       
print(patternB_list)     # last element is a number of row, it should be actually e.g. ([1,2,5], 0)    

[array([0, 1, 0]), array([3, 4, 2]), array([6, 7, 5])]


In [512]:
patternC_list = []

for i in range(len(footprint)):
    tab_tmp2 = []
    for j in range(len(footprint)):
        if(footprint[j][i] == 1):
            tab_tmp2.append(j)
    if(len(tab_tmp2) > 1):
        tab_tmp2.append(i)
        tab_tmp2 = np.asarray(tab_tmp2)
        patternC_list.append(tab_tmp2)
       
print(patternC_list)     # last element is a number of row, it should be actually e.g. ([0,7], 1)    

[array([3, 4, 5]), array([6, 7, 8])]


In [513]:
for i in range(len(patternB_list)):
    tupl = (patternB_list[i][0], patternB_list[i][1])
    tupl2 = (patternB_list[i][0], patternB_list[i][2])
    if tupl in dependency_list:
        dependency_list.remove(tupl)
    if tupl2 in dependency_list:
        dependency_list.remove(tupl2)  

print(dependency_list)

[(2, 3), (2, 4), (3, 5), (4, 5), (5, 6), (5, 7), (6, 8), (7, 8)]


In [514]:
for i in range(len(patternC_list)):
    tupl = (patternC_list[i][0], patternC_list[i][2])
    tupl2 = (patternC_list[i][1], patternC_list[i][2])
    if tupl in dependency_list:
        dependency_list.remove(tupl)
    if tupl2 in dependency_list:
        dependency_list.remove(tupl2)  

print(dependency_list)

[(2, 3), (2, 4), (5, 6), (5, 7)]


In [515]:
for i in range(len(footprint)):
    for j in range(len(footprint)):
        if(footprint[i][j] == 3):
            dependency_list.remove((i, j))

print(dependency_list)

[(2, 3), (2, 4), (5, 6), (5, 7)]


In [516]:
tab_of_tasks = []

for elem in range(max_task):
    tab_of_tasks.append(elem)
            
print(tab_of_tasks)

[0, 1, 2, 3, 4, 5, 6, 7]


In [517]:
dependency_list.extend(loop_list)
print(dependency_list)
print(patternB_list)
print(patternC_list)

[(2, 3), (2, 4), (5, 6), (5, 7), (1, 2), (2, 1)]
[array([0, 1, 0]), array([3, 4, 2]), array([6, 7, 5])]
[array([3, 4, 5]), array([6, 7, 8])]


In [1]:
import bpmn_python.bpmn_diagram_rep as diagram

import graphviz
 
bpmn_graph = diagram.BpmnDiagramGraph()
bpmn_graph.create_new_diagram_graph(diagram_name="diagram1")
process_id = bpmn_graph.add_process_to_diagram()

TabError: inconsistent use of tabs and spaces in indentation (graph.py, line 792)

In [519]:
tab = {}
tab_help=[]

#beginning
[start_id, _] = bpmn_graph.add_start_event_to_diagram(process_id, start_event_name="start_event")
if(len(Tinput) > 1):  
    [exclusive_gate_id, _] = bpmn_graph.add_exclusive_gateway_to_diagram(process_id, gateway_name="X")
    for elem in Tinput:
        if elem not in tab_help:
            tab[elem] = bpmn_graph.add_task_to_diagram(process_id, task_name=elem)
            tab_help.append(elem)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, start_id, exclusive_gate_id)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, exclusive_gate_id, tab[Tinput[0]][0])
    bpmn_graph.add_sequence_flow_to_diagram(process_id, exclusive_gate_id, tab[Tinput[1]][0])
else:
    tab[Tinput[0]] = bpmn_graph.add_task_to_diagram(process_id, task_name=Tinput[0])
    bpmn_graph.add_sequence_flow_to_diagram(process_id, start_id, tab[Tinput[0]][0])
    tab_help.append(Tinput[0])
 
# connections a#(b,c)
for elem in patternB_list:
    exclusive_gate_id = bpmn_graph.add_exclusive_gateway_to_diagram(process_id, gateway_name="X")
    for val in elem:
        if(val not in tab_help):
            tab[val] = bpmn_graph.add_task_to_diagram(process_id, task_name=val)
            tab_help.append(val)  
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[elem[0]][0], exclusive_gate_id[0])
    #if(elem[0] == elem[1]):
    bpmn_graph.add_sequence_flow_to_diagram(process_id, exclusive_gate_id[0], tab[elem[1]][0])
    bpmn_graph.add_sequence_flow_to_diagram(process_id, exclusive_gate_id[0], tab[elem[2]][0])     

# connections (b,c)#d
for elem in patternC_list:
    [exclusive_gate_id, _] = bpmn_graph.add_exclusive_gateway_to_diagram(process_id, gateway_name="X")
    for val in elem:
        if(val not in tab_help):
            tab[val] = bpmn_graph.add_task_to_diagram(process_id, task_name=val)
            tab_help.append(val)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[elem[0]][0], exclusive_gate_id)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[elem[1]][0], exclusive_gate_id)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, exclusive_gate_id, tab[elem[2]][0])    

# tasks
for elem in dependency_list:
    if(elem[0] not in tab_help):
        tab[elem[0]] = bpmn_graph.add_task_to_diagram(process_id, task_name=elem[0])
        tab_help.append(elem[0])
    if(elem[1] not in tab_help):
        tab[elem[1]] = bpmn_graph.add_task_to_diagram(process_id, task_name=elem[1])
        tab_help.append(elem[1])
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[elem[0]][0], tab[elem[1]][0])     
    
# end
if(len(Toutput) > 1):
    [exclusive_gate_id, _] = bpmn_graph.add_exclusive_gateway_to_diagram(process_id, gateway_name="X")
    for elem in Toutput:
        if elem not in tab_help: 
            tab[elem] = bpmn_graph.add_task_to_diagram(process_id, task_name=elem)
            tab_help.append(elem)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[Toutput[0]][0], exclusive_gate_id)
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[Toutput[1]][0], exclusive_gate_id)
    [end_id, _] = bpmn_graph.add_end_event_to_diagram(process_id, end_event_name="end_event")
    bpmn_graph.add_sequence_flow_to_diagram(process_id, exclusive_gate_id, end_id)
else:
    if Toutput[0] not in tab_help:
        tab[Toutput[0]] = bpmn_graph.add_task_to_diagram(process_id, task_name=Toutput[0])
    [end_id, _] = bpmn_graph.add_end_event_to_diagram(process_id, end_event_name="end_event")
    bpmn_graph.add_sequence_flow_to_diagram(process_id, tab[Toutput[0]][0], end_id)       
    
visualizer.bpmn_diagram_to_png(bpmn_graph, file_name = 'model_example')